# Derivation of `RWC_alam()`
*Last updated on 2022-05-16 by Arthur Ryman*

## Introduction

The goal of this notebook is to create a simple test case that reproduces the following
exception.

In [1]:
from acmpy.globals import ACM_set_defaults
ACM_set_defaults(0)

In [2]:
from acmpy.hamiltonian_data import RWC_alam

try:
    RWC_alam(15, -3, 2)
except ValueError as ve:
    print(ve)

max() arg is an empty sequence


## `RWC_alam()`

The code listings follow.

### Maple

```
# # The following procedure RWC_alam returns values of the ACM parameters
# # (anorm,lambda), which are "optimal" in the cases of the RWC Hamiltonians.
# # This seeks the minimal value of RWC_expt, given above, by solving
# # for a turning point.
# # The fourth parameter is for seniority v, which is 0 for the
# # analysis given in [WR2015], but in the case of more general v,
# # (for L=0 (so 3\v) and no spherical dependence in the potential),
# # the 9/4 factor in the first term of (B.16) is replaced by
# # the more general (v+3/2)^2.
#
# RWC_alam:=proc(B::constant,c1::constant,c2::constant,v::nonnegint:=0,$)
#   local RWC1,RWC2,muf,aa0,vshft,A;
#
#   vshft:=(2*v+3)^2:  # This is 9 for the v=0 case.
#
#   if evalf(c1)<0 then
#           # Here lambda is a function of aa (i.e. a^2).
#           # There is always one positive solution in this case
#           # (in fact, I've never found other real solns).
#
#     # We use mu=2(lambda-1) where lambda is given by (B11) via (B15).
#
#     muf:=(aa) -> sqrt( vshft + (aa*c1/c2)^2 ):
#
#     # The following is the derivative of (B16) noting that
#     # d(mu)/d(aa)=aa*(c1/c2)^2/mu.  But multiplied by 4*aa^3*B*mu.
#
#     RWC2:=(aa,mu) -> (c1/c2)^2 * (-vshft*aa^5/mu^2
#                                     + aa^3*B^2*c1 + aa^2*B^2*c2*(mu+3))
#                        + aa^3*(2*mu+vshft)
#                        - B^2*mu*(mu+2)*(aa*c1+c2*(mu+4)):
#
#     aa0:=max(fsolve(RWC2(A,muf(A))=0,A)):
#     return [sqrt(aa0),1+muf(aa0)/2]:
#
#   else   # (Here lambda is constant)
#          # There is always 1 positive solution in this case
#          # (fsolve produces real solns.) and possibly two others that
#          # are negative. Use max to exclude them.
#
#     RWC1:=(aa) -> aa^3 - B^2*c1*aa - (2*v+7)*B^2*c2:
#
#     aa0:=max(fsolve(RWC1(A)=0,A)):
#     return [sqrt(aa0),2.5]:
#   fi:
#
# end:
```

### Python
```

def RWC_alam(B: float, c1: float, c2: float, v: nonnegint = 0
             ) -> tuple[float, float]:
    require_nonnegint('v', v)

    vshft: int = (2 * v + 3) ** 2

    A: Symbol = symbols('A', real=True)
    aa0: float
    if c1 < 0:

        def muf(aa: Expr) -> Expr:
            return sqrt(vshft + (aa * c1 / c2) ** 2)

        def RWC2(aa: Expr, mu: Expr) -> Expr:
            return (c1 / c2) ** 2 * (-vshft * aa ** 5 / mu ** 2
                                     + aa ** 3 * B ** 2 * c1  + aa **2 * B ** 2 * c2 * (mu + 3)) \
                   + aa ** 3 * (2 * mu + vshft) \
                   - B ** 2 * mu * (mu + 2) * (aa * c1 + c2 * (mu + 4))

        aa0 = max(float(aa) for aa in solveset(RWC2(A, muf(A)), A, domain=S.Reals))
        return math.sqrt(aa0), float(1 + muf(S(aa0)) / 2)

    else:

        def RWC1(aa: Expr) -> Expr:
            return aa ** 3 - B ** 2 * c1 * aa - (2 * v + 7) * B ** 2 * c2

        aa0 = max(float(aa) for aa in solveset(RWC1(A), A, domain=S.Reals))
        return math.sqrt(aa0), 2.5

```

## Mathematical Definitions

Here are the mathematical formulae encoded in SymPy.

In [3]:
from sympy import *

B = Symbol('B', real=True, positive=True)
c1 = Symbol('c1', real=True, negative=True)
c2 = Symbol('c2', real=True, positive=True)
lambda0 = Symbol('lambda0', real=True, positive=True)
a = Symbol('a', real=True, positive=True)
A = Symbol('A', real=True, positive=True)
E = Symbol('E', real=True)
beta0 = Symbol('beta0', real=True, nonnegative=True)

In [4]:
A_a = a ** 2
A_a

a**2

In [5]:
E1_A_lambda0_B = A / (2 * B) * (1 + 9 / (4 * (lambda0 - 1)))
E1_A_lambda0_B

A*(1 + 9/(4*lambda0 - 4))/(2*B)

In [6]:
E2_A_lambda0_B_c1 =  B / (2 * A) * c1 * lambda0
E2_A_lambda0_B_c1

B*c1*lambda0/(2*A)

In [7]:
E3_A_lambda0_B_c2 = B / (2 * A ** 2) * c2 * lambda0 * (lambda0 + 1)
E3_A_lambda0_B_c2

B*c2*lambda0*(lambda0 + 1)/(2*A**2)

In [8]:
E_A_lambda0_B_c1_c2 = E1_A_lambda0_B + E2_A_lambda0_B_c1 + E3_A_lambda0_B_c2
E_A_lambda0_B_c1_c2

A*(1 + 9/(4*lambda0 - 4))/(2*B) + B*c1*lambda0/(2*A) + B*c2*lambda0*(lambda0 + 1)/(2*A**2)

We only consider the case $c_1 < 0$.

In [9]:
beta0_c1_c2 = sqrt(-c1 / (2 * c2))
beta0_c1_c2

sqrt(2)*sqrt(-c1)/(2*sqrt(c2))

In [10]:
lambda0_A_beta0 = 1 + sqrt(Rational(9, 4) + (A * beta0 ** 2) ** 2)
lambda0_A_beta0

sqrt(A**2*beta0**4 + 9/4) + 1

In [11]:
lambda0_A_c1_c2 = lambda0_A_beta0.subs(beta0, beta0_c1_c2)
lambda0_A_c1_c2

sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1

In [12]:
mu = Symbol('mu', real=True, positive=True)
mu_A_c1_c2 = sqrt(9 + (A * c1 / c2) ** 2)
mu_A_c1_c2

sqrt(A**2*c1**2/c2**2 + 9)

In [13]:
mu_lambda0 = 2 * (lambda0 - 1)
mu_lambda0

2*lambda0 - 2

In [14]:
lambda0_mu = mu / 2 + 1
lambda0_mu

mu/2 + 1

In [15]:
mu_A_c1_c2 = mu_lambda0.subs(lambda0, lambda0_A_c1_c2)
mu_A_c1_c2

2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)

In [16]:
simplify(mu_A_c1_c2)

sqrt(A**2*c1**2 + 9*c2**2)/c2

In [17]:
E_A_lambda0_B_c1_c2

A*(1 + 9/(4*lambda0 - 4))/(2*B) + B*c1*lambda0/(2*A) + B*c2*lambda0*(lambda0 + 1)/(2*A**2)

In [18]:
E_A_mu_B_c1_c2 = E_A_lambda0_B_c1_c2.subs(lambda0, lambda0_mu)
E_A_mu_B_c1_c2

A*(1 + 9/(2*mu))/(2*B) + B*c1*(mu/2 + 1)/(2*A) + B*c2*(mu/2 + 1)*(mu/2 + 2)/(2*A**2)

In [19]:
mu_A_c1_c2

2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)

In [20]:
Dmu_DA_A_c1_c2 = mu_A_c1_c2.diff(A)
Dmu_DA_A_c1_c2

A*c1**2/(2*c2**2*sqrt(A**2*c1**2/(4*c2**2) + 9/4))

In [21]:
Dmu_DA_A_c1_c2 * mu_A_c1_c2

A*c1**2/c2**2

In [22]:
E_A_B_c1_c2 = E_A_mu_B_c1_c2.subs(mu, mu_A_c1_c2)
E_A_B_c1_c2

A*(1 + 9/(4*sqrt(A**2*c1**2/(4*c2**2) + 9/4)))/(2*B) + B*c1*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)/(2*A) + B*c2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 2)/(2*A**2)

In [23]:
DE_DA_A_B_c1_c2 = E_A_B_c1_c2.diff(A)
DE_DA_A_B_c1_c2

-9*A**2*c1**2/(32*B*c2**2*(A**2*c1**2/(4*c2**2) + 9/4)**(3/2)) + B*c1**3/(8*c2**2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)) + (1 + 9/(4*sqrt(A**2*c1**2/(4*c2**2) + 9/4)))/(2*B) + B*c1**2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)/(8*A*c2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)) + B*c1**2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 2)/(8*A*c2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)) - B*c1*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)/(2*A**2) - B*c2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 2)/A**3

In [24]:
E_A_mu_B_c1_c2

A*(1 + 9/(2*mu))/(2*B) + B*c1*(mu/2 + 1)/(2*A) + B*c2*(mu/2 + 1)*(mu/2 + 2)/(2*A**2)

In [25]:
mu_A_c1_c2

2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)

In [26]:
Dmu_DA_A_mu_c1_c2 = (c1 / c2) ** 2 * A / mu
Dmu_DA_A_mu_c1_c2

A*c1**2/(c2**2*mu)

In [27]:
Z = E_A_mu_B_c1_c2.diff(A) + E_A_mu_B_c1_c2.diff(mu) * Dmu_DA_A_mu_c1_c2
Z

A*c1**2*(-9*A/(4*B*mu**2) + B*c1/(4*A) + B*c2*(mu/2 + 1)/(4*A**2) + B*c2*(mu/2 + 2)/(4*A**2))/(c2**2*mu) + (1 + 9/(2*mu))/(2*B) - B*c1*(mu/2 + 1)/(2*A**2) - B*c2*(mu/2 + 1)*(mu/2 + 2)/A**3

In [28]:
Z1 = Z * 4 * A ** 3 * B * mu
Z1

A**3*B*mu*(4*A*c1**2*(-9*A/(4*B*mu**2) + B*c1/(4*A) + B*c2*(mu/2 + 1)/(4*A**2) + B*c2*(mu/2 + 2)/(4*A**2))/(c2**2*mu) + 2*(1 + 9/(2*mu))/B - 2*B*c1*(mu/2 + 1)/A**2 - 4*B*c2*(mu/2 + 1)*(mu/2 + 2)/A**3)

In [29]:
RWC2_A_mu_B_c1_c2 = (c1/c2) ** 2 *(-9 * A ** 5 / mu ** 2 + \
                      A ** 3 * B ** 2 * c1 + A ** 2 * B ** 2 * c2 * (mu + 3)) \
+ A ** 3 * (2 * mu + 9) \
- B ** 2 * mu * (mu + 2) * (A * c1 + c2 * (mu + 4))
RWC2_A_mu_B_c1_c2

A**3*(2*mu + 9) - B**2*mu*(mu + 2)*(A*c1 + c2*(mu + 4)) + c1**2*(-9*A**5/mu**2 + A**3*B**2*c1 + A**2*B**2*c2*(mu + 3))/c2**2

In [30]:
simplify(RWC2_A_mu_B_c1_c2 - Z1)

0

## The Simplified Test Case

In summary, we have the following two expressions.

In [31]:
mu_A_c1_c2

2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)

In [32]:
RWC2_A_mu_B_c1_c2

A**3*(2*mu + 9) - B**2*mu*(mu + 2)*(A*c1 + c2*(mu + 4)) + c1**2*(-9*A**5/mu**2 + A**3*B**2*c1 + A**2*B**2*c2*(mu + 3))/c2**2

Specialize these expressions to $c_1 = -3$ and $c_2 = 2$.

In [33]:
mu_A = simplify(mu_A_c1_c2.subs([(c1, -3), (c2, 2)]))
mu_A

3*sqrt(A**2 + 4)/2

In [34]:
RWC2_A_mu_B = RWC2_A_mu_B_c1_c2.subs([(c1, -3), (c2, 2)])
RWC2_A_mu_B

-81*A**5/(4*mu**2) - 27*A**3*B**2/4 + A**3*(2*mu + 9) + 9*A**2*B**2*(mu + 3)/2 - B**2*mu*(mu + 2)*(-3*A + 2*mu + 8)

Solve the case $B=14$.

In [35]:
F_A_B = RWC2_A_mu_B.subs(mu, mu_A)
F_A_B

-9*A**5/(A**2 + 4) - 27*A**3*B**2/4 + A**3*(3*sqrt(A**2 + 4) + 9) + 9*A**2*B**2*(3*sqrt(A**2 + 4)/2 + 3)/2 - 3*B**2*sqrt(A**2 + 4)*(3*sqrt(A**2 + 4)/2 + 2)*(-3*A + 3*sqrt(A**2 + 4) + 8)/2

In [36]:
F_A_14 = F_A_B.subs(B, 14)
F_A_14

-9*A**5/(A**2 + 4) + A**3*(3*sqrt(A**2 + 4) + 9) - 1323*A**3 + 882*A**2*(3*sqrt(A**2 + 4)/2 + 3) - 294*sqrt(A**2 + 4)*(3*sqrt(A**2 + 4)/2 + 2)*(-3*A + 3*sqrt(A**2 + 4) + 8)

In [37]:
F_A_15 = F_A_B.subs(B, 15)
F_A_15

-9*A**5/(A**2 + 4) + A**3*(3*sqrt(A**2 + 4) + 9) - 6075*A**3/4 + 2025*A**2*(3*sqrt(A**2 + 4)/2 + 3)/2 - 675*sqrt(A**2 + 4)*(3*sqrt(A**2 + 4)/2 + 2)*(-3*A + 3*sqrt(A**2 + 4) + 8)/2

In [38]:
sol_14 = solveset(F_A_14, A, domain=S.Reals)
sol_14

{CRootOf(x**12 + 1188*x**10 - 6664*x**9 - 418020*x**8 - 865536*x**7 - 6516688*x**6 + 2700096*x**5 - 36973440*x**4 + 61101824*x**3 - 92198400*x**2 + 147517440*x - 86051840, 1)}

In [ ]:
sol_15 = solveset(F_A_15, A, domain=S.Reals)
sol_15

In [ ]:
import math

for x in sol_14:
    aa = float(x)
    a0 = math.sqrt(aa)
    print(aa, a0)

In [ ]:
for x in sol_15:
    aa = float(x)
    a0 = math.sqrt(aa)
    print(aa, a0)

## Conclusion

I cannot reproduce the problem this way!

Next, work with the `RWC_alam()` function in isolation and debug it.
Simplify it to find where the error occurs.